# Spooky Author Identification

You may (or may not) have heard of the below 3 authors:

- **Mary Wollstonecraft Shelley** was an English novelist famous for writing "Frankenstein" in 1818.
- **Edgar Allen Poe** was an American writer and poet famous for his writing including "The Raven" (1845) and "The Tell-Tale Heart" (1843)
- **H.P. Lovecraft** was an American horror fiction writer who is known for writing "The Call of Cthulhu" (1928)

... also all three have been referenced in "The Simpsons" which some of us may consider to be more noteworthy.

We will generally call these 3 writers "Spooky" Authors. For those who are really familiar with the above authors they may be able to read a short passage of writing and determine which of the 3 authors wrote the piece. Our task today will be to write a computer program that will do just that; given a short passage of writing will correctly identify its author.

*The data and inspiration for this work come from: https://www.kaggle.com/c/spooky-author-identification

## Algorithm

Our approach is fairly simple (and perhaps overly naive). We will examine a bunch of quotes from each of the above authors and we will use this data to determine how frequently they use each word in their writing. 

Then for an unclassified quote, for each possible author, we will compute the product of frequencies for the words in that quote. The author having the highest product is guessed to be the author of the quote.

Lastly we will compare our guess against the actual author and track our success rate.

Consider a simple example:

- Author 1 Quotes: `[ "banana banana orange", "orange black yellow", "banana yellow"]`

- Author 2 Quotes: `[ "gold banana", "texas tea", "gold tea", "orange pekoe"]`

Author 1 frequencies: "banana": `3/8`, "orange": `2/8`, "black": `1/8`, "yellow": `2/8`


Author 2 frequencies: "gold": `2/8`, "banana": `1/8`, "texas": `1/8`, "tea": `2/8`, "orange": `1/8`, "pekoe": `1/8`

Now for the quote: *"orange banana"*

We consider that Author 1 has product: `2/8 (orange) * 3/8 (banana) = 6/64` 

Author 2 has product: `1/8 (orange) * 1/8 (banana) = 1/64`

Therefore we guess that **Author 1** is the Author of the quote since `6/64 > 1/64`

## Groups

Today's case study will be completed in groups of 3 - 5 students each.

Please assemble into groups of 3-5 by asking your neighbours to join the group.

# Task 1: Introduction (3 minutes)

1. Introduce yourselves
2. Appoint one person as the **paper recorder** - this person should have a pen and a paper
3. Appoint one person as the **computer recorder** - this person should have a connected device (laptop preferable)
4. Appoint one person as the **communicator** - this person will speak to the instructor, or class and/or write stuff on the board if required
5. The computer recorder should open this notebook and then proceed to download the data (next 2 cells below)

   a. make sure code ai suggestions is turned off so we can all practice our coding with dictionaries
6. Save the notebook to your local google drive 


## Data Description

As is common in machine learning frameworks, we will be given a **training** data set and a **testing** data set.

Each dataset consists of a single .csv file.

A CSV file is a common-separated-values format where each line in the file is a row in a table, where each column is separated by commas. In our case each line will contain 3 columns of data, an id, a quote, and the author.

Example:

`"id10633","It was the beating of the old man's heart.","EAP"`

In the above there is an id: `"id10633"`, a quote: `"It was the beating of hte old man's heart."`, and the author: `"EAP"` denoting Edgar Allen Poe.

Each author will be denoted by initials: `EAP` for Edgar Allen Poe, `MWS` for Mary Wollstonecraft Shelley and `HPL` for H.P. Lovecraft.

We will use the `train.csv` dataset to build up a model for the frequencies of words each of our 3 authors use. Then we will see how this simple model can identify each model when we show it the test.csv dataset.

* Note: mostly we will be practicing some of our programming skills like dictionary use rather than building a quality NLP (natural language processing) model to accomplish this task. But you can always explore the actual approaches people used at the above kaggle link. 


In [ ]:
# save this case study to your own drive
# the following code will download the data files required for this case study

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

examples_path = "https://github.com/andrewgodbout/F2025CS1910/raw/refs/heads/main/CaseStudies/data.zip"


download(examples_path)
!unzip -o data.zip



## csv Module

From last week we learned about reading from files and spliting based on delimiters (like commas ,) but ...

Rather than writing our own csv parsing code we will use the csv module. This module has a `DictReader` class that is capable of 
converting each row of the csv file into its own dictionary. In our case it will have `id`, `quote` and `author` keys:

example of a quote attributed to Edgar Allan Poe (EAP):
```python
{ 'id' : '001',
  'quote' : 'nevermore',
  'author': 'EAP'
}



In [ ]:
#use the csv module to load the data
import csv
with open('train.csv') as csvfile:

    #DictReader will populate a dictionary for each line in the csv
    reader = csv.DictReader(csvfile, fieldnames=["id", "quote", "author"])

    #create a counter so we can just print the first two records in the file
    cnt = 0
    #each row is a dictionary
    for row in reader:
        if cnt < 2:
            print(row)
        cnt += 1
    
    print("\n\ntotal lines read:", reader.line_num)

# Group Task 2: Understanding the data (5 minutes)

*The group paper recorder should put a paper in the middle of the work space so all can see*

In your groups discuss the above code and below questions and record your answers on paper.

1. **How many different authors are their in our dataset?**

2. **The training dataset has 19479 records each made up of 3 fields: `id`, `quote` and `author`, which of the fields is least important to our goal?**

In the above code the variable `row` is initialized as a dictionary with keys: `id`, `quote` and `author` for each record/line in the `train.csv` data file. Recall we want to know how frequently words are used by each author -> we want to count how often they use each word.

3. **What do we want to do with each of these rows?**

i.e., in plain english what do we want to do to the `row` variable on each iteration of the loop?

Things to consider:

**1. How do we access the data inside `row`?**


**2. How do we want our data organized?**

We want the count of words for each author

Perhaps a dictionary for each author where the key is the word and the value is the count for that word?
```python
#edgar allan poe wrote about ravens and beating hearts:
poe_dict = { 'raven' : 29, 'heart' : 22, ...}
```

**3. How should we process each row?**


**4. Do we need to do any data pre-processing?**

Does punctuation impact things? 

Do want to bother with words like "the"?

Does Capitalization impact things?

**5. Other considerations**



# Task 3 Pseudo Code on Paper (5 minutes) 

Write out a high level pseudo-code solution on paper for how to process the training data. High level means avoid the details. We are looking for less than 6 high level steps for how to process our train.csv file, i.e., how to determine the frequency of word use for each author.

```







```



## Pre-processing 

We will clean the quote by pre-precessing it 

1. To avoid issues with punctuation, i.e., "hello," being a different word from "hello" we will remove all punctuation from the quotes. 
2. To avoid issues with capitalization, i.e., "hello" being different from "Hello" we will convert all quotes to lower case
3. To avoid issues with common words, i.e., analyzing "and", "I" or "a" we will remove these `STOP` words from the quote





## Task 4: Implement the below clean_quote method (5 minutes) 

In [ ]:
import csv
import string

#Task 4-1: Ask generative ai to suggest some stop words in a python list (including those from 1800 and 1900s writing. 
#          Populate them into the below list

stop_words = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
    "be", "because", "been", "before", "being", "below", "between", "both", "but", "by",
    "could",
    "did", "do", "does", "doing", "down", "during",
    "each",
    "few", "for", "from", "further",
    "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how",
    "i", "if", "in", "into", "is", "it", "its", "itself",
    "let's",
    "me", "more", "most", "my", "myself",
    "nor", "not",
    "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own",
    "same", "she", "should", "so", "some", "such",
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too",
    "under", "until", "up",
    "very",
    "was", "we", "were", "what", "when", "where", "which", "while", "who", "whom", "why", "with", "would",
    "you", "your", "yours", "yourself", "yourselves",
"thou", "thee", "thy", "thine", "ye", "hath", "doth", "dost", "art", "shalt", "wilt", "hast", "wert",
    "whilst", "ere", "oft", "nay", "yea", "saith", "unto", "wherefore", "whence", "whither"
]

print(string.punctuation)

def clean_quote(quote :str, stop_words: list[str]) -> list[str]:
    """ For a given quote clean the quote in 3 ways
    1. remove punctuation
    2. convert to lower case
    3. remove stop words 

    @param: quote (string) the given quote to clean
    @param: stop_words (list of strings) a list of all of the words that should be removed from quote

    @return a list of strings representing the words in the cleaned quote
    """

    #step 1 convert the quote to lower case
    #step 2: remove punctuation (keep only characters that aren't punctuation)
    #        note: that the string module has a list of common punctuation 
    #              in a variable: string.punctuation

    #step 1 and 2 are given for you:
    clean = ''.join([char.lower() for char in quote if char not in string.punctuation])
    

    #step 3: keep only the non-stop words
    #        note: if you split a string by " " (space) 
    #              you get a list of the "words" in that string you can iterate over



    
    #step 4: return a list of words that have been cleaned

    #remove this placeholder code
    return []


In [ ]:
#once you have completed the above clean_quote the following code should produce no errors
#don't forget to "run" the above cell before running this cell

good_words = clean_quote("You say, \"Goodbye\" and I say, \"Hello, hello, HELLO\"!", stop_words)
assert "HELLO" not in good_words, "capital HELLO test failed"
assert "i" not in good_words, "stop word test failed"


## Task 5: Implement Count Words and Convert to Frequency methods ( 5 minutes )

`count_words` is only a couple of lines just be careful when dealing with words that might not yet be in the dictionary

`convert_to_freqs` does not take as input the total number of words, how can we obtain this easily from input_dict?


In [2]:
def count_words(good_words :str, author_dict: dict[str, int]) -> None:
    """ Given a list of words and a dictionary of word counts
        increment the count for each word in the list

        @param good_words a list of cleaned words
        @param author_dict a dictionary with keys as strings (words) and 
                           value int representing the count of occurrences of that word
        @return None nothing is returned
    """
    
    #hint use the .get method to avoid issues when incrementing a word for the first time
    

def convert_to_freqs(input_dict: dict[str, int]) -> dict[str, float]:
    """ For a given input dictionary of words (str) and their respective counts (int)
       return a new dictionary with the words (str) and their frequency of occurrence (float)

       the frequency of a word is the count of that word divided by the total number of words (sum of the values).
    
        @param input_dict a dictionary of words and respective counts
        @return dictionary with the same keys as input_dict but having values as the 
                frequency of occurrence of those words
    """

    #remove this placeholder code
    return {}

In [ ]:
# when you complete your methods this code should run properly

author_dict = {}
good_words = ["say", "goodbye", "say", "hello", "hello", "hello"]
count_words(good_words, author_dict)
assert "hello" in author_dict and author_dict["hello"] == 3, "hello test failed expecting count=3"
assert "i" not in author_dict, "stop word test failed, expecting \"i\" is a stop word"
assert len(author_dict) == len(["say", "goodbye", "hello"]), "number of keys should be 3 test failed"

freq_dict = convert_to_freqs(author_dict)

assert len(freq_dict) == len(author_dict), "len should match output and input"
assert abs(freq_dict.get("goodbye", 0) - author_dict.get("goodbye", 0.1)/sum(author_dict.values())) < 0.00001, "check freq calculation"

print("all tests complete")

In [ ]:
""" 
Glue code 
The following code will process our training dataset by iterating over the records and calling our above methods
"""


#create 3 dictionaries to store the count of each word for each author
eap = {}
mws = {}
hpl = {}


with open('train.csv') as csvfile:

    #DictReader will populate a dictionary for each line in the csv
    reader = csv.DictReader(csvfile, fieldnames=["id", "quote", "author"])

    #create a counter so we can just print the first two records in the file
    cnt = 0
    for row in reader:

        #task 4: clean the input and give back a list of cleaned words
        good_words = clean_quote(row['quote'], stop_words)
        
        if cnt < 2:
            print("original:", row['quote'])
            print ("cleaned quote:", good_words)
        cnt += 1

        author = row['author']

        #set the correct dictionary
        author_dict = eap
        if author == "MWS":
            author_dict = mws

        elif author == "HPL":
            author_dict = hpl

        #Task 5 count of the cleaned words in an author_dictionary
        count_words(good_words, author_dict)
        
    print("\n\ntotal lines read:", reader.line_num)


#task 5: convert the dictionary of counts into a dictionary of frequencies
eap_f = convert_to_freqs(eap)
mws_f = convert_to_freqs(mws)
hpl_f = convert_to_freqs(hpl)

print("total EAP words:", len(eap))
print("total MWS words:",len(mws))
print("total HPL words:",len(hpl))


## Task 6 Discussion and Pseudo-code (5 minutes)

We have processed the training dataset and have 3 dictionaries giving the frequency of use of an authors words. Our algorithm is to compare an unseen quote to our model (3 dictionaries of author word frequencies) and then predict the author of the unseen quote by computing the product of frequencies for the words in the unseen quote. 

Our product of frequencies may end up multiplying many small numbers together, which can be problematic, to avoid this we will only multiply together the *X* words having the largest frequency for that author.

Discuss how to implement this with your group and form pseudo code on paper for your solution

Discussion points:

1. Can we reuse any of our work from prior tasks?

2. What happens if the frequency for a word is zero? What happens if a word shows up twice in the same quote?

3. How can we limited our product calculation to include only the *k* (depth) largest values?



## Task 7 Implement the product_freq method (7 minutes)

In [ ]:
# convert to frequencies

def product_freq(clean_quote:list[str], author_freq_dict:str, depth:int) -> float:
    """
    For a given cleaned quote (list of strings) and an author_freq_dict of words and their 
    frequency of use, compute the product of frequencies for the words having the depth (int)
    largest frequencies

    In the case that a word in clean_quote is not present in the author_freq_dict then small_number should be its frequency.

    @param clean_quote: a list of words 
    @param author_freq_dict: A dictionary of words and their associated frequency of use for a particular author
    @param depth: The number of words to use in the product calculation (only the depth-most words), if depth is larger than 
                  the length of the quote then only len(clean_quote) words are used in the product. Depth is always larger than 0

                  
    @return a floating point number giving the product of word frequencies for the maximum depth largest frequencies.
    
    """

    small_number = 0.0000001

    #remove this placeholder code
    return 0

In [ ]:
#once the above is complete the following tests should pass
clean_quote = ["worst", "episode", "ever"]
author_freq_dict = { "lisa": 0.1, "needs": 0.2, "braces":0.2, "episode": 0.35, "ever":0.15}

threshold = 0.000001

#depth longer than quote
prod = product_freq(clean_quote, author_freq_dict, 4)
assert abs(prod - .15 * 0.35) < threshold, f"product frequency expecting {.15*.35} got {prod}"

#depth longer than author_freq_dict
prod = product_freq(clean_quote, author_freq_dict, 3)
assert abs(prod - .15 * 0.35) < threshold, f"product frequency expecting {.15*.35} got {prod}"

#depth equals clean length
prod = product_freq(clean_quote, author_freq_dict, 2)
assert abs(prod - .15 * 0.35) < threshold, f"product frequency expecting {.15*.35} got {prod}"

#depth shorter than quote
prod = product_freq(clean_quote, author_freq_dict, 1)
assert abs(prod - 0.35) < threshold, f"product frequency expecting .35 got {prod}"

print("above test results")


In [ ]:
# Do the testing

with open('test.csv') as csvfile:

    #DictReader will populate a dictionary for each line in the csv
    reader = csv.DictReader(csvfile, fieldnames=["id", "quote", "author"])

    #create a counter so we can just print the first two records in the file
    cnt = 0
    correct_cnt = 0
    #experiment by changing the depth
    depth = 10
    for row in reader:

        cnt += 1
        author = row['author']

        #reuse task 4
        good_words = clean_quote(row['quote'], stop_words)        

        #task 7 for each author
        prod_eap = (product_freq(good_words, eap_f, depth), "EAP")
        prod_mws = (product_freq(good_words, mws_f, depth), "MWS")
        prod_hpl = (product_freq(good_words, hpl_f, depth), "HPL")

        #the largest product is the winner (our guess of the author)
        winner = max(prod_eap, prod_mws, prod_hpl)
        print(f"{author}{good_words[0:3]}  {winner}")

        if winner[1] == row['author']:
            correct_cnt += 1
            
    print(correct_cnt)
    print(cnt)
    print("accuracy:", correct_cnt/cnt)

## Class Checkin
What accuracy level did everyone achieve?

# Experiment

What happens in the above if depth is changed?

*depth = 1, 10, 25, or 100, for example*

What is the role of `small_number`? 

*Why not just multiply by zero when we encounter words that are not in our training set?*

# Further reading

If you found this interesting, natural language processing is an area of computer science / data science. Our approach is quite simple but not unrelated to cosine-similarity for comparing the "closeness" of things or TF-IDF (term frequency inverse document frequency) which is a way to represent word distributions.




# share

Share your group's version of the case study with your teammates

# on your own 

Can you modify the above code to determine accuracy for each individual author? 

Which author had the highest accuracy?